In [1]:
%cd ..
%pwd
# %cd code
# should be /code
# 
%load_ext autoreload
%autoreload 2

/home/oli/Research/Joe/agent-goals/code


In [2]:
from pgmpy.models import BayesianNetwork,  FactorGraph

from pgmpy.readwrite import BIFReader
# reader = BIFReader('data/alarm.bif')
reader = BIFReader('data/asia.bif')
#reader.get_edges()
alarm = reader.get_model()

In [3]:
from pdg import PDG
from pdg.rv import Variable as Var
from pdg.dist import CPT, RawJointDist as RJD
from pdg.alg import interior_pt as ip

import numpy as np
import cvxpy as cp

## Tests for `ip.cvx_opt_*` 

In [5]:
alarm_PDG = PDG.from_BN(alarm)

In [ ]:
# VERY DANGEROUS!!
#ip.cvx_opt_clusters(alarm_PDG)
#

Cs = list(alarm_PDG.to_markov_net().to_junction_tree().nodes())
print(list(Cs))
print(sorted([len(c) for c in Cs], reverse=True))


import numpy as np
print([np.prod([len(alarm_PDG.vars[X]) for X in C]) for C in Cs])

# prob = ip.cvx_opt_clusters(alarm_PDG, Cs, also_idef=False, dry_run=True, verbose=True)

# print(ns, sum(ns))

In [ ]:
smoke,lung,bronc = alarm_PDG.vars['smoke'], alarm_PDG.vars['lung'], alarm_PDG.vars['bronc']
dysp,either = alarm_PDG.vars['dysp'], alarm_PDG.vars['either']

alarm_PDG.factor_product()[smoke,lung,bronc]

In [6]:
full_F = ip.cvx_opt_clusters(alarm_PDG, varname_clusters=[[V.name for V in alarm_PDG.varlist]], 
    also_idef=False,verbose=True).marginals[0]
# full_F[smoke, lung, bronc]
full_T = ip.cvx_opt_clusters(alarm_PDG, varname_clusters=[[V.name for V in alarm_PDG.varlist]],
     also_idef=True, verbose=True).marginals[0]


marginalsF = ip.cvx_opt_clusters(alarm_PDG, also_idef=False, verbose=True).marginals
marginalsT = ip.cvx_opt_clusters(alarm_PDG, also_idef=True, verbose=True).marginals

joint_F = ip.cvx_opt_joint(alarm_PDG, also_idef=False, verbose=True)
joint_T = ip.cvx_opt_joint(alarm_PDG, also_idef=True, verbose=True)

[(0, ('asia', 'smoke', 'bronc', 'either', 'dysp', 'lung', 'tub', 'xray'))]
                                     CVXPY                                     
                                    v1.1.18                                    
(CVXPY) Sep 13 09:50:28 PM: Your problem has 292 variables, 9 constraints, and 0 parameters.
(CVXPY) Sep 13 09:50:28 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Sep 13 09:50:28 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Sep 13 09:50:28 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Sep 13 09:50:28 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Sep 13 09:5

In [ ]:
full_T[smoke,lung,bronc]

In [7]:
from pgmpy.inference.ExactInference import BeliefPropagation, VariableElimination

bp = BeliefPropagation(alarm)
ve = VariableElimination(alarm)

bp2 = BeliefPropagation(alarm.to_markov_model())
ve2 = BeliefPropagation(alarm.to_markov_model())

bp.calibrate()
bp2.calibrate()

# takes 48 seconds;
# ... or 1m 2sec
# takes 5GB RAM. 

In [ ]:
bp.get_cliques()

In [ ]:
[f.values.size for f in bp.get_clique_beliefs().values()]

In [ ]:
fg = alarm_PDG.to_FG(drop_joints=True)
mm = alarm.to_markov_model()
print("Same factors...")
print(' > ',[f.scope() for f in mm.factors], ';')
print(' > ',[fg.scope(f) for f in fg.factors])
print()

for (discf, f) in zip(mm.factors, fg.factors):
    assert set(discf.scope()) == set(fg.scope(f))
    
    

In [ ]:
tcpd = alarm.get_cpds("dysp")
# print(tcpd.get_evidence())
# print(tcpd.values[0,1])
# tcpd.store_state_names(['bronc','either'],[2,2], {'bronc' : ['+','-'], 'either':['y', 'n']})
print(tcpd)
print(tcpd.variables[:0])
print(tcpd.values[0][0,1])


p = CPT.from_pgmpy(tcpd)
print(p.nfrom)
p

# alarm_PDG[ alarm_PDG.vars['dysp'] | alarm_PDG.vars['either'] & alarm_PDG.vars['bronc'] ]

In [ ]:


vevars = [alarm_PDG.vars[n] for n in ve.variables]

# ve_vals = ve.query([V.name for V in reversed(alarm_PDG.varlist)]).values
Q = ve.query(ve.variables)
ve_vals = Q.values
# print(ve.)

ve_rjd_1 = RJD(ve_vals, vevars)
ve_rjd_2 = RJD(ve_vals, alarm_PDG.varlist)
ve_rjd_3 = RJD(ve_vals, list(reversed(vevars)))
ve_rjd = RJD(ve_vals, [alarm_PDG.vars[n] for n in Q.variables])


print(ve.query(['asia']))

print('[1]      vevars:',[V.name for V in ve_rjd_1.varlist])
print('[2] pdg varlist:',[V.name for V in ve_rjd_2.varlist])
print('[3]  ve reversed:',[V.name for V in ve_rjd_3.varlist])
print('[4]  query order:',[V.name for V in ve_rjd.varlist])


print(ve_rjd_1[alarm_PDG.vars['asia']].to_numpy())
print(ve_rjd_2[alarm_PDG.vars['asia']].to_numpy())
print(ve_rjd_3[alarm_PDG.vars['asia']].to_numpy())
print(ve_rjd[alarm_PDG.vars['asia']].to_numpy())


# jointT[alarm_PDG.vars['smoke']]

In [8]:
mu_ghalf_joint = ip.cccp_opt_joint(alarm_PDG, 0.5)
mu_ghalf_cluster = ip.cccp_opt_clusters(alarm_PDG, 0.5).marginals

mu_g2_joint = ip.cccp_opt_joint(alarm_PDG, 2, max_iters=50)
mu_g2_cluster = ip.cccp_opt_clusters(alarm_PDG, 2, max_iters=50).marginals

(vex)    p1 	 0.5
(vex)    p2 	 0.5
(vex)    p3 	 0.5
(vex)    p4 	 0.5
(vex)    p5 	 0.5
(vex)    p6 	 0.5
(vex)    p7 	 0.5
(vex)    p8 	 0.5
obj:  -1.270042071510602e-08 		 tv distance 0.2864558213944233
obj:  -1.270042071510602e-08 		 tv distance 0.0
no clusters given; using pgmpy junction tree to find some.
FOUND:  [('tub', 'xray', 'dysp', 'asia', 'smoke'), ('tub', 'xray', 'dysp', 'bronc', 'lung', 'smoke'), ('tub', 'xray', 'dysp', 'bronc', 'lung', 'either')]
[(0, ('tub', 'xray', 'dysp', 'asia', 'smoke')), (1, ('tub', 'xray', 'dysp', 'bronc', 'lung', 'smoke')), (2, ('tub', 'xray', 'dysp', 'bronc', 'lung', 'either'))]
[(('tub', 'xray', 'dysp', 'asia', 'smoke'), ('tub', 'xray', 'dysp', 'bronc', 'lung', 'smoke'), {'weight': 64}), (('tub', 'xray', 'dysp', 'bronc', 'lung', 'smoke'), ('tub', 'xray', 'dysp', 'asia', 'smoke'), {'weight': 32}), (('tub', 'xray', 'dysp', 'bronc', 'lung', 'smoke'), ('tub', 'xray', 'dysp', 'bronc', 'lung', 'either'), {'weight': 64}), (('tub', 'xray', 'dysp', 'b

In [9]:

mu_g10_joint = ip.cccp_opt_joint(alarm_PDG, 10, max_iters=50)
mu_g10_cluster = ip.cccp_opt_clusters(alarm_PDG, 10, max_iters=50).marginals

(cave)   p1 	 -9
(cave)   p2 	 -9
(cave)   p3 	 -9
(cave)   p4 	 -9
(cave)   p5 	 -9
(cave)   p6 	 -9
(cave)   p7 	 -9
(cave)   p8 	 -9
obj:  10.87485734110794 		 tv distance 0.013924444531127362
obj:  3.3150646520245246 		 tv distance 0.015426520294980785
obj:  2.332745621670149 		 tv distance 0.019353229883441875
obj:  1.633889755460359 		 tv distance 0.021280703385089708
obj:  1.148589417568111 		 tv distance 0.021478653419957416
obj:  0.8144520237273909 		 tv distance 0.02051361513819773
obj:  0.5839503628634368 		 tv distance 0.0189534401246884
obj:  0.4236442062362862 		 tv distance 0.017166839196232186
obj:  0.31089429317584916 		 tv distance 0.01536284852538572
obj:  0.23061133447346194 		 tv distance 0.013658609430258595
obj:  0.1727391690532798 		 tv distance 0.012101089297860579
obj:  0.13053025502977533 		 tv distance 0.01070628330751433
obj:  0.09940806512808464 		 tv distance 0.009469088431524192
obj:  0.07622929830153069 		 tv distance 0.008375173668425356
obj:  0.058813

In [16]:
np.set_printoptions(precision=3)

def toVars(*varnames):
    return [alarm_PDG.vars[v] for v in varnames]

VVn = ('dysp', 'bronc', 'either')
# VVn = ('dysp',)
# VVn = ('either', 'bronc')


VV = toVars(*VVn)
clique = next(c for c in bp.get_cliques() if all(v in c for v in VVn))

bp_cluster_rjd = RJD(bp.get_clique_beliefs()[clique].values, [alarm_PDG.vars[n] for n in clique])
# bp2_cluster_rjd = RJD(bp2.get_clique_beliefs()[clique].values, [alarm_PDG.vars[n] for n in clique])

print('belief prop\t\t', bp_cluster_rjd[VV].to_numpy().reshape(-1))
# print('BP2\t\t',  bp2_cluster_rjd[VV].to_numpy().reshape(-1))
print('variable elimination\t', ve.query(list(VVn)).values.reshape(-1))
Q = ve.query(ve.variables)
ve_rjd = RJD(Q.values, [alarm_PDG.vars[n] for n in Q.variables] )
# print(vals.shape, vals.sum())
print('var elim2\t\t', ve_rjd[VV].to_numpy().reshape(-1))
# print()

print('factor-product\t\t', alarm_PDG.factor_product()[VV].to_numpy().reshape(-1))
# print('factor-graph\t\t', alarm_PDG.to_FG().dist[VV].to_numpy().reshape(-1))
# print('fg (dropjoint)\t', fg.dist[VV].to_numpy().reshape(-1))
# print('fg (dropjoint)\t\t', alarm_PDG.to_FG(drop_joints=True).dist[VV].to_numpy().reshape(-1))
print()


cluster_idx = next(i for (i,m) in enumerate(marginalsF) if all(v in m.varlist for v in VV))

print('cvx-cluster\t\t', marginalsF[cluster_idx][VV].to_numpy().reshape(-1))
print('cvx-big-clust\t\t', full_F[VV].to_numpy().reshape(-1))
print('cvx-joint\t\t', joint_F[VV].to_numpy().reshape(-1))
print('cvx-cluster \t+idef\t',marginalsT[cluster_idx][VV].to_numpy().reshape(-1))
print('cvx-1big-clust \t+idef\t', full_T[VV].to_numpy().reshape(-1))
print('cvx-joint-T \t+idef\t', joint_T[VV].to_numpy().reshape(-1))
print()


# cluster_idx = next(i for (i,m) in enumerate(mu_g2_cluster) if all(v in m.varlist for v in VV))
print('cccp-cluster\t (γ=.5)\t', mu_ghalf_cluster[cluster_idx][VV].to_numpy().reshape(-1))
print('cccp-joint\t (γ=.5)\t', mu_ghalf_joint[VV].to_numpy().reshape(-1))
print('cccp-cluster\t (γ=2)\t', mu_g2_cluster[cluster_idx][VV].to_numpy().reshape(-1))
print('cccp-joint\t (γ=2)\t', mu_g2_joint[VV].to_numpy().reshape(-1))
print('cccp-cluster\t (γ=10)\t', mu_g10_cluster[cluster_idx][VV].to_numpy().reshape(-1))
print('cccp-joint\t (γ=10)\t', mu_g10_joint[VV].to_numpy().reshape(-1))
print()



















Finding Elimination Order: : 100%|██████████| 4/4 [00:00<00:00, 167.85it/s]




0it [00:00, ?it/s]

belief prop		 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]
variable elim	 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]
var elim2	 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]
factor-product		 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]

cvx-cluster		 [0.031 0.332 0.019 0.052 0.003 0.083 0.008 0.471]
cvx-big-clust		 [0.033 0.331 0.018 0.052 0.004 0.083 0.008 0.472]
cvx-joint		 [0.033 0.331 0.018 0.052 0.004 0.083 0.008 0.472]
cvx-cluster 	+idef	 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]
cvx-1big-clust 	+idef	 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]
cvx-joint-T 	+idef	 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]

cccp-cluster	 (γ=.5)	 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]
cccp-joint	 (γ=.5)	 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]
cccp-cluster	 (γ=2)	 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]
cccp-joint	 (γ=2)	 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]
cccp-cluster	 (γ=10)	 [0.033 0.331 0.021 0.053 0.004 0

## Tests for the CCCP for when $\gamma > 0$.

In [ ]:
np.set_printoptions(precision=3)

gamma1 = ip.cccp_opt_joint(alarm_PDG, 10, max_iters=500)

print('fg (dropjoint)\t', alarm_PDG.to_FG(drop_joints=True).dist[VV].to_numpy().reshape(-1))
print('gamma=1 \t', gamma1[VV].to_numpy().reshape(-1))

In [ ]:
print("Inc: ", alarm_PDG.Inc(gamma1), "\tIDef: ", alarm_PDG.IDef(gamma1))
# alarm_PDG[alarm_PDG.vars['either'] | alarm_PDG.vars['lung'] & al
# arm_PDG.vars['tub']]

In [ ]:
gamma1[alarm_PDG.vars['either'] | alarm_PDG.vars['lung'],alarm_PDG.vars['tub']]


In [ ]:
ip._dup2shape(cp.Variable(10), (2,3,5), (0,2))

In [ ]:
ip.cccp_opt_clusters(alarm_PDG, gamma=1.5)